<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/03.%20Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-retrieval} -->

# Understanding Retrieval Question Answering

### Setup

In [1]:
!pip install -Uqqq rich openai==0.27.2 tiktoken wandb langchain unstructured tabulate pdf2image chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import os, random
from pathlib import Path
import tiktoken
from getpass import getpass
from rich.markdown import Markdown
import openai

In [7]:
from google.colab import userdata
# Recuperar a chave API do segredo salvo
openai_api_key = userdata.get('OPENAI_API_KEY')

# Definir a chave API como uma variável de ambiente
os.environ['OPENAI_API_KEY'] = openai_api_key
openai.api_key = openai_api_key

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [8]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


## Langchain

[LangChain](https://docs.langchain.com/docs/) is a framework for developing applications powered by language models. We will use some of its features in the code below. Let's start by configuring W&B tracing.

In [9]:
# we need a single line of code to start tracing langchain with W&B
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

# wandb documentation to configure wandb using env variables
# https://docs.wandb.ai/guides/track/advanced/environment-variables
# here we are configuring the wandb project name
os.environ["WANDB_PROJECT"] = "llmapps"

## Parsing documents

We will use a small sample of markdown documents in this notebook. Let's find them and make sure we can stuff them into the prompt. That means they may need to be chunked and not exceed some number of tokens.

In [4]:
MODEL_NAME = "text-davinci-003"
# MODEL_NAME = "gpt-4"

Let's grab some sample data, here we will download a folder containing some markdown files from our docs.

In [10]:
!git clone https://github.com/wandb/edu.git

Cloning into 'edu'...
remote: Enumerating objects: 4766, done.
remote: Counting objects: 100% (1650/1650), done.
remote: Compressing objects: 100% (725/725), done.
remote: Total 4766 (delta 1186), reused 1196 (delta 913), pack-reused 3116 (from 1)
Receiving objects: 100% (4766/4766), 41.95 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (2598/2598), done.


In [12]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36


In [13]:
from langchain.document_loaders import DirectoryLoader

def find_md_files(directory):
    "Find all markdown files in a directory and return a LangChain Document"
    dl = DirectoryLoader(directory, "**/*.md")
    return dl.load()

documents = find_md_files('edu/llm-apps-course/docs_sample/')
len(documents)

11

In [14]:
# We will need to count tokens in the documents, and for that we need the tokenizer
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)

In [15]:
# function to count the number of tokens in each document
def count_tokens(documents):
    token_counts = [len(tokenizer.encode(document.page_content)) for document in documents]
    return token_counts

count_tokens(documents)

[750, 661, 2560, 421, 1089, 4191, 3090, 1647, 309, 1193, 2121]

We will use `LangChain` built in `MarkdownTextSplitter` to split the documents into sections. Actually splitting `Markdown` without breaking syntax is not that easy. This splitter strips out syntax.
- We can pass the `chunk_size` param and avoid lenghty chunks.
- The `chunk_overlap` param is useful so you don't cut sentences randomly. This is less necessary with `Markdown`

The `MarkdownTextSplitter` also takes care of removing double line breaks and save us some tokens that way.

In [16]:
from langchain.text_splitter import MarkdownTextSplitter

md_text_splitter = MarkdownTextSplitter(chunk_size=1000)
document_sections = md_text_splitter.split_documents(documents)
len(document_sections), max(count_tokens(document_sections))

(97, 402)

let's look at the first section

In [17]:
Markdown(document_sections[0].page_content)

import Tabs from '@theme/Tabs'; import TabItem from '@theme/TabItem';                                              

Tags                                                                                                               

Tags can be used to label runs with particular features that might not be obvious from the logged metrics or       
Artifact data -- this run's model is in_production, that run is preemptible, this run represents the baseline.     

How to add tags                                                                                                    

You can add tags to a run when it is created: wandb.init(tags=["tag1", "tag2"]) .                                  

You can also update the tags of a run during training (e.g. if a particular metrics crosses a pre-defined          
threshold):                                                                                                        

                                                                                                                   
                                                                                                                   
 ...                                                                                                               
                                                                                                                   
 if current_loss < threshold: run.tags = run.tags + ("release_candidate",)                                         
                                                                                                                   


## Embeddings

Let's now use embeddings with a vector database retriever to find relevant documents for a query.

In [18]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# We will use the OpenAIEmbeddings to embed the text, and Chroma to store the vectors
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(document_sections, embeddings)

<ipython-input-18-7c4ea4ad0ace>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


We can create a retriever from the db now, we can pass the `k` param to get the most relevant sections from the similarity search

In [19]:
retriever = db.as_retriever(search_kwargs=dict(k=3))

In [20]:
query = "How can I share my W&B report with my team members in a public W&B project?"
docs = retriever.get_relevant_documents(query)

<ipython-input-20-0c6e59c8888f>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


In [21]:
# Let's see the results
for doc in docs:
    print(doc.metadata["source"])

edu/llm-apps-course/docs_sample/collaborate-on-reports.md
edu/llm-apps-course/docs_sample/collaborate-on-reports.md
edu/llm-apps-course/docs_sample/teams.md


## Stuff Prompt

We'll now take the content of the retrieved documents, stuff them into prompt template along with the query, and pass into an LLM to obtain the answer.

In [22]:
from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

context = "\n\n".join([doc.page_content for doc in docs])
prompt = PROMPT.format(context=context, question=query)

Use langchain to call openai chat API with the question

In [23]:
from langchain.llms import OpenAI

llm = OpenAI()
response = llm.predict(prompt)
Markdown(response)

<ipython-input-23-dab957efc3a1>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
<ipython-input-23-dab957efc3a1>:4: LangChainDeprecationWarning: The method `BaseLLM.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)


To share your W&B report with your team members in a public W&B project, you can either provide their email        
addresses or copy the magic link and share it with them. They will need to log into Weights & Biases to view the   
report if they are invited by email, but they will not need to log in if they are given the magic link. Shared     
reports are view-only.

## Using Langchain

Langchain gives us tools to do this efficiently in few lines of code. Let's do the same using `RetrievalQA` chain.

In [24]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)
result = qa.run(query)

Markdown(result)

<ipython-input-24-82df38eb5826>:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)


To share your W&B report with your team members in a public W&B project, you can follow these steps:               

  1 First, make sure you are logged in to your W&B account.                                                        
  2 Go to the report that you want to share with your team members.                                                
  3 Click on the Share button on the upper right hand corner.                                                      
  4 You will see two options: "Provide an email account" and "Copy the magic link."                                
  5 If you want to invite specific team members, you can provide their email accounts. They will need to log in to 
    W&B to view the report.                                                                                        
  6 If you want to share the report with all your team members, you can copy the magic link and send it to them.   
    They do not need to log in to W&B to view the report.                                                          
  7 Once your team members have access to the report, they can view it in a read-only mode. They will not be able  
    to make any edits to the report unless you give them edit access.                                              
  8 To give your team members edit access to the report, you can toggle the permissions between edit and view      
    access for each team member under the team project settings. Only the administrator or member who created the  
    report can do this.                                                                                            
  9 Your team

In [25]:
import wandb
wandb.finish()